# Proses Pengumpulan Data Gas NO₂ (Nitrogen Dioksida) Dari Citra Satelit Sentinel-5P Melalui Platform OpenEO Untuk Wilayah Kabupaten Pamekasan, Jawa Timur.

1. Import dan Setup Awal 

import openeo
import pandas as pd
import os
import time

Penjelasan:

Program mengimpor pustaka yang dibutuhkan, yaitu:
- openeo: library untuk koneksi dan pemrosesan data satelit via API Copernicus.
- pandas: untuk manipulasi dan analisis data hasil (CSV, tabel, dll).
- os: untuk akses file di folder lokal.
- time: digunakan untuk memberi jeda saat menunggu job selesai.
Kemudian, program membuat koneksi ke server OpenEO Copernicus Data Space dan melakukan autentikasi menggunakan OpenID Connect agar dapat mengakses dataset Sentinel-5P.

2. Koneksi ke Copernicus Data Space

connection = openeo.connect("openeo.dataspace.copernicus.eu").authenticate_oidc()

Penjelasan:

Menghubungkan Python ke OpenEO server milik Copernicus.
- authenticate_oidc(): autentikasi login via OpenID Connect (biasanya muncul pop-up login akun Copernicus).

3. Menentukan Area of Interest (AOI)

aoi = {
    "type": "Polygon",
    "coordinates": [
        [
            [113.45, -7.12],  # barat laut
            [113.45, -7.20],  # barat daya
            [113.55, -7.20],  # tenggara
            [113.55, -7.12],  # timur laut
            [113.45, -7.12],  # kembali ke titik awal
        ]
    ],
}

Penjelasan:

Program mendefinisikan wilayah pengamatan (AOI) dalam format GeoJSON Polygon.
Koordinat yang digunakan mencakup area di sekitar Kota Pamekasan dan sekitarnya dengan batas:
- Barat: 113.45° BT
- Timur: 113.55° BT
- Utara: −7.12° LS
- Selatan: −7.20° LS
AOI ini digunakan untuk memfilter area pengambilan data Sentinel-5P agar hanya mencakup wilayah yang relevan.

4. Mengambil Data Sentinel-5P

s5p = connection.load_collection(
    "SENTINEL_5P_L2",
    spatial_extent={
        "west": 113.45,
        "south": -7.20,
        "east": 113.55,
        "north": -7.12,
    },
    temporal_extent=["2024-10-01", "2025-10-31"], 
    bands=["NO2"],
)

Penjelasan:

- load_collection: mengambil data citra dari koleksi satelit tertentu.
- "SENTINEL_5P_L2": dataset Sentinel-5P level 2 (NO₂, CO, O₃, dll).
- spatial_extent: batas area (sesuai AOI).
- temporal_extent: rentang waktu (1 Oktober 2024 s/d 31 Oktober 2025).
- bands=["NO2"]: hanya mengambil kanal gas Nitrogen Dioksida (NO₂).
Program memanggil dataset SENTINEL_5P_L2 dan mengambil band “NO₂” (Nitrogen Dioksida) dalam rentang waktu 1 Oktober 2024 hingga 31 Oktober 2025.
Parameter spatial_extent dan temporal_extent memastikan hanya data dari wilayah dan waktu tersebut yang diambil.

5. Masking Nilai Negatif

def mask_invalid(x):
    return x < 0

s5p_masked = s5p.mask(s5p.apply(mask_invalid))

Penjelasan:

- Beberapa data satelit bisa bernilai negatif akibat noise atau kesalahan pengukuran.
- Fungsi ini memfilter nilai negatif (tidak valid) agar tidak ikut dihitung.
Program membuat fungsi mask_invalid() untuk menandai nilai-nilai NO₂ yang tidak valid (negatif). Nilai negatif tidak masuk akal secara ilmiah, sehingga data tersebut dihapus (dimasking).

6. Agregasi Temporal (Rata-rata Harian)

daily_mean = s5p_masked.aggregate_temporal_period(period="day", reducer="mean")

Penjelasan:

- Mengubah data per-pixel per-waktu menjadi rata-rata harian.
- period="day": hitung rata-rata untuk setiap hari.
- reducer="mean": fungsi agregasi yang digunakan adalah rata-rata.
Data citra satelit Sentinel-5P dikumpulkan secara harian (daily) dengan menghitung nilai rata-rata NO₂ setiap hari (aggregate_temporal_period(period="day", reducer="mean")).
Langkah ini mengubah data spasial harian menjadi time series yang mudah dianalisis.

7. Agregasi Spasial (Rata-rata Wilayah AOI)

daily_mean_aoi = daily_mean.aggregate_spatial(geometries=aoi, reducer="mean")

Penjelasan:

- Menghitung rata-rata NO₂ di seluruh area Pamekasan untuk setiap tanggal.
- Hasilnya: 1 nilai NO₂ per hari.
Setelah agregasi temporal, program menghitung rata-rata spasial di dalam area AOI (aggregate_spatial).
Hasilnya adalah satu nilai rata-rata NO₂ per hari untuk seluruh wilayah Pamekasan.

8. Eksekusi Batch Job

job = daily_mean_aoi.execute_batch(out_format="CSV")
print("\nMenunggu job OpenEO selesai...")

Penjelasan:
- execute_batch(): mengirim perintah ke server OpenEO agar memproses semua data.
- out_format="CSV": hasil akhir disimpan dalam format CSV.
Seluruh proses di atas dijalankan sebagai batch job di server Copernicus.
Program menunggu hingga status job berubah menjadi finished, dengan pengecekan berkala setiap 15 detik. Jika job gagal, program akan menampilkan pesan error.

9. Monitoring Status Job

while True:
    status = job.describe()["status"]
    print(f"Status: {status}")
    if status == "finished":
        break
    elif status == "error":
        raise RuntimeError("Job gagal")
    time.sleep(15)

Penjelasan:

- Mengecek status job setiap 15 detik.
- Jika status berubah jadi "finished", proses lanjut.
- Jika "error", program berhenti dengan pesan kesalahan.
Setelah job selesai, hasil perhitungan dikemas dalam format CSV dan diunduh ke folder lokal bernama "no2_results".
Program memastikan setidaknya satu file CSV berhasil diunduh.

10. Membaca dan Membersihkan Data

if "date" in df.columns:
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df = df.sort_values("date").reset_index(drop=True)
    df.to_csv("timeseries.csv", index=False)

    print(f"\nData berhasil dikumpulkan")
    print(f"  - Total records: {len(df)}")
    print(f"  - Periode: {df['date'].min()} hingga {df['date'].max()}")
    print(f"  - File: timeseries.csv")
    print(f"\nData pertama (5 baris):")
    print(df.head())
else:
    print("\n⚠️ Tidak ditemukan kolom 'date' dalam hasil CSV. Cek format hasil dari OpenEO.") 

Penjelasan:

File CSV hasil unduhan dibaca menggunakan pandas. Kolom date diubah menjadi format tanggal (datetime) dan data diurutkan berdasarkan waktu.
File hasil akhir disimpan sebagai timeseries.csv.

11. Output Informasi

Program menampilkan ringkasan hasil, termasuk:
- Jumlah total record data NO₂ harian,
- Rentang tanggal pengamatan,
- Nama file hasil (timeseries.csv), serta
- Lima baris pertama dari data.


In [1]:
# Kode

pip install openeo

# ==============================================================
# LANGKAH 1: PENGUMPULAN DATA - Sentinel-5P NO2 di Pamekasan, Madura
# ==============================================================

import openeo
import pandas as pd
import os
import time

print("="*70)
print("LANGKAH 1: PENGUMPULAN DATA ")
print("="*70)

# 1. Koneksi ke Copernicus Data Space
connection = openeo.connect("openeo.dataspace.copernicus.eu").authenticate_oidc()

# 2. Area of Interest (AOI) - Sekitar Kabupaten Pamekasan, Jawa Timur
aoi = {
    "type": "Polygon",
    "coordinates": [
        [
            [113.45, -7.12],  # barat laut
            [113.45, -7.20],  # barat daya
            [113.55, -7.20],  # tenggara
            [113.55, -7.12],  # timur laut
            [113.45, -7.12],  # kembali ke titik awal
        ]
    ],
}

# 3. Load data Sentinel-5P NO2
s5p = connection.load_collection(
    "SENTINEL_5P_L2",
    spatial_extent={
        "west": 113.45,
        "south": -7.20,
        "east": 113.55,
        "north": -7.12,
    },
    temporal_extent=["2024-10-01", "2025-10-31"], 
    bands=["NO2"],
)

# 4. Mask nilai negatif
def mask_invalid(x):
    return x < 0

s5p_masked = s5p.mask(s5p.apply(mask_invalid))

# 5. Agregasi temporal (harian)
daily_mean = s5p_masked.aggregate_temporal_period(period="day", reducer="mean")

# 6. Agregasi spasial (mean dalam AOI)
daily_mean_aoi = daily_mean.aggregate_spatial(geometries=aoi, reducer="mean")

# 7. Jalankan batch job
job = daily_mean_aoi.execute_batch(out_format="CSV")
print("\nMenunggu job OpenEO selesai...")

while True:
    status = job.describe()["status"]
    print(f"Status: {status}")
    if status == "finished":
        break
    elif status == "error":
        raise RuntimeError("Job gagal")
    time.sleep(15)

# 8. Unduh hasil
results = job.get_results()
results.download_files("no2_results")

# 9. Baca file CSV
csv_files = [f for f in os.listdir("no2_results") if f.endswith(".csv")]
if len(csv_files) == 0:
    raise FileNotFoundError("Tidak ada file CSV yang diunduh dari hasil OpenEO.")

df = pd.read_csv(os.path.join("no2_results", csv_files[0]))

# 10. Data preprocessing
if "date" in df.columns:
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df = df.sort_values("date").reset_index(drop=True)
    df.to_csv("timeseries.csv", index=False)

    print(f"\nData berhasil dikumpulkan")
    print(f"  - Total records: {len(df)}")
    print(f"  - Periode: {df['date'].min()} hingga {df['date'].max()}")
    print(f"  - File: timeseries.csv")
    print(f"\nData pertama (5 baris):")
    print(df.head())
else:
    print("\n⚠️ Tidak ditemukan kolom 'date' dalam hasil CSV. Cek format hasil dari OpenEO.") 

SyntaxError: invalid syntax (3651421891.py, line 3)